In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_equipment_path ='/kaggle/input/2022-ukraine-russian-war/russia_losses_equipment.csv'
data_personnel_path ='/kaggle/input/2022-ukraine-russian-war/russia_losses_personnel.csv'

## Goal
This is the dataset that describes Equipment Losses & Death Toll & Military Wounded & Prisoner of War of russians in 2022 Ukraine russia War.

The data is commutative. It means each new record is accumulated data from previous days. 

The goal of the notebook is to represent data on a daily based. It is very simple but could be very useful for analysis. Please, use the code snipet to convert data.

## Data, Last 20 days

In [ ]:
df_equipment = pd.read_csv(data_equipment_path)
df_equipment['date'] = pd.to_datetime(df_equipment['date'])
df_equipment = df_equipment.set_index(['date', 'day', 'greatest losses direction'])
# df_equipment = df_equipment.drop('greatest losses direction', axis=1)
df_equipment = df_equipment.diff().fillna(df_equipment).fillna(0).astype(int).reset_index()
df_equipment.tail(20)

In [ ]:
df_personnel = pd.read_csv(data_personnel_path)
df_personnel['date'] = pd.to_datetime(df_personnel['date'])
df_personnel = df_personnel.set_index(['date', 'day', 'personnel*'])
df_personnel = df_personnel.diff().fillna(df_personnel).fillna(0).astype(int).reset_index()
df_personnel.tail(20)

## Plot, example

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import dates
from dateutil.relativedelta import relativedelta

fig_author = 'Petro Ivaniuk'
fig_date = pd.Timestamp('today').date()
period_months = 2
column_equipment = 'drone'
column_date = 'date'

df_ = df_equipment[[column_date, column_equipment]]
date_last = df_.iloc[-1][column_date]
date_first = date_last - relativedelta(months=+period_months)
df_ = df_[df_[column_date]>date_first].copy()

label_list = df_[column_date]
number_list = df_[column_equipment]

fig = plt.figure(figsize=(21, 7), dpi=300)
ax = fig.add_subplot()
ax.bar(label_list, number_list)

for patch in ax.patches:
    ax.text(
        patch.get_x()+patch.get_width()/2, 
        patch.get_height()+0.1,
        patch.get_height() if patch.get_height()!=0 else '', 
        ha="center", 
        fontsize=12
    )
    
ax.tick_params(axis='x', labelrotation=90)
ax.xaxis.set_major_locator(dates.DayLocator(interval=2))

ax.tick_params(labelleft=False)   
ax.yaxis.set_ticks_position('none')
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.annotate('Created by {}, {}.'.format(fig_author, fig_date), xy=(0.8, 0.85), xycoords='figure fraction',)

plt.title('Armed Forces of Ukraine in action, russian {} losses'.format(column_equipment), fontsize=20)
plt.tight_layout()
plt.savefig('{}.png'.format(column_equipment))
plt.show()